# Recap train at scale

# 1) Explain concepts of incremental fit by chunks

<img src='https://wagon-public-datasets.s3.amazonaws.com/data-science-images/07-ML-OPS/train_by_chunk.png'>

# 2) Explain code solution for `main_local.train()`

```python
def train():
    """
    Training on the full (already preprocessed) dataset, by loading it
    chunk-by-chunk, and updating the weight of the model for each chunks.
    Save model, compute validation metrics on a holdout validation set that is
    common to all chunks.
    """
    print("\n ⭐️ use case: train")

    # Validation Set: Load a validation set common to all chunks and create X_val, y_val
    data_val_processed_path = os.path.abspath(os.path.join(
        LOCAL_DATA_PATH, "processed", f"val_processed_{VALIDATION_DATASET_SIZE}.csv"))

    
    data_val_processed = pd.read_csv(
        data_val_processed_path,
        header=None,
        dtype=DTYPES_PROCESSED_OPTIMIZED
        ).to_numpy()

    X_val = data_val_processed[:, :-1]
    y_val = data_val_processed[:, -1]
    

    # Iterate on the full training dataset chunk per chunks.
    # Break out of the loop if you receive no more data to train upon!
    model = None
    chunk_id = 0
    metrics_val_list = []  # store each metrics_val_chunk

    while (True):
        print(f"loading and training on preprocessed chunk n°{chunk_id}...")

        # Load chunk of preprocess data and create (X_train_chunk, y_train_chunk)
        
        path = os.path.abspath(os.path.join(
            LOCAL_DATA_PATH, "processed", f"train_processed_{DATASET_SIZE}.csv"))

        try:
            data_processed_chunk = pd.read_csv(
                    path,
                    header=None,
                    skiprows=(chunk_id * CHUNK_SIZE),
                    nrows=CHUNK_SIZE,
                    dtype=DTYPES_PROCESSED_OPTIMIZED,
                    ).to_numpy()

        except pd.errors.EmptyDataError:
            data_processed_chunk = None  # end of data

        # Break out of while loop if we have no data to train upon
        if data_processed_chunk is None:
            break

        X_train_chunk = data_processed_chunk[:, :-1]
        y_train_chunk = data_processed_chunk[:, -1]
        

        # Train a model incrementally and print validation metrics for this chunk
        learning_rate = 0.001
        batch_size = 256
        
        if model is None:
            model = initialize_model(X_train_chunk)
            model = compile_model(model, learning_rate)

        model, history = train_model(model,
                                     X_train_chunk,
                                     y_train_chunk,
                                     batch_size,
                                     validation_data=(X_val, y_val))
        metrics_val_chunk = np.min(history.history['val_mae'])
        metrics_val_list.append(metrics_val_chunk)
        print(metrics_val_chunk)
        

        chunk_id += 1

    # Save model and training params
    params = dict(
        learning_rate=learning_rate,
        batch_size=batch_size,
        incremental=True,
        chunk_size=CHUNK_SIZE)

    metrics_val_mean_all_chunks = np.mean(np.array(metrics_val_list))
    metrics = dict(mean_val=metrics_val_mean_all_chunks)

    save_model(model, params=params, metrics=metrics)

    print("✅ model trained and saved")
```

# 3) 💻 Tensorflow tricks to partial fit without manual chunks


**📚Resources📚**
- tf CSV guide: https://www.tensorflow.org/guide/data#consuming_csv_data
- tf CSV tuto: https://www.tensorflow.org/tutorials/load_data/csv
- tf Datasets https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/data.ipynb#scrollTo=x5z5B11UjDTd

**Import packages**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import os

In [ ]:
import taxifare
taxifare.__file__

In [ ]:
from taxifare.ml_logic.params import LOCAL_DATA_PATH

In [ ]:
data_processed_path_small = os.path.join(LOCAL_DATA_PATH, "processed","train_processed_100K.csv")
data_processed_path = os.path.join(LOCAL_DATA_PATH, "processed","train_processed_500K.csv")

**Import model**

We'll copy paste it below to make it more explicit

In [ ]:
BATCH_SIZE=265

In [ ]:
def build_model():
    
    reg = regularizers.l1_l2(l2=0.005)
    model = Sequential()
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(100, activation="relu", kernel_regularizer=reg))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(rate=0.1))
    model.add(layers.Dense(50, activation="relu", kernel_regularizer=reg))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(rate=0.1))
    model.add(layers.Dense(10, activation="relu"))
    model.add(layers.BatchNormalization(momentum=0.99))  # use momentum=0 for to only use statistic of the last seen minibatch in inference mode ("short memory"). Use 1 to average statistics of all seen batch during training histories.
    model.add(layers.Dropout(rate=0.1))
    model.add(layers.Dense(1, activation="linear"))
    
    optimizer = keras.optimizers.Adam(learning_rate= 0.001)
    model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=["mae"])
    
    return model


In [ ]:
es = EarlyStopping(monitor="val_loss",
                       patience=2,
                       restore_best_weights=True,
                       verbose=0)

## 3.1) If data fit in memory 😇

In [ ]:
df_small = pd.read_csv(data_processed_path_small)
df_small

In [ ]:
features = df_small.drop(columns=['65']).to_numpy()
target = df_small[['65']].to_numpy()

In [ ]:
print(features.shape)
print(target.shape)
n_samples = features.shape[0]
n_features = features.shape[1]

### a) passing numpy arrays

In [ ]:
model = build_model()

model.fit(x=features, y=target, batch_size=BATCH_SIZE, validation_split=0.3, callbacks=[es], epochs=10)

### b) passing `datasets`

In [ ]:
ds = tf.data.Dataset.from_tensor_slices((features, target))
ds = ds.batch(BATCH_SIZE)  # Set batch size

In [ ]:
ds.element_spec

In [ ]:
# First sample: feature_1, target_1
f1, t1 = next(iter(ds))
(f1.shape, t1.shape)

In [ ]:
model = build_model()
model.fit(ds, epochs=5)

## 3.2) If data is too large to fit in memory ? 🧐 

💡 Use `make_csv_dataset` helper

More info on this tutorial https://www.tensorflow.org/tutorials/load_data/csv

In [ ]:
ds = tf.data.experimental.make_csv_dataset(
    data_processed_path,
    batch_size=BATCH_SIZE,
    header=False,
    column_names=list(df_small.columns),
    label_name='65',
    num_epochs=1,
    ignore_errors=True)

In [ ]:
ds.element_spec

In [ ]:
feat1, target1 = next(iter(ds))

In [ ]:
target1.shape

In [ ]:
def stack(x):
    return tf.stack([x[f'{i}'] for i in range(65)], axis=1)

stack(feat1).shape

In [ ]:
ds = ds.map(lambda x,y: (stack(x),y))

In [ ]:
model = build_model()
model.fit(ds, epochs=5)